In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 14.1 MB/s eta 0:00:00


In [ ]:
!git clone <repo_url>

In [ ]:
!rm -r /content/drive/MyDrive/object_detection/colab/yolov5/runs/danger_detection_v1_0/latest_results

In [ ]:
cd /content/drive/MyDrive/object_detection/colab/yolov5

/content/drive/MyDrive/KNU_Project2/object_detection/colab/yolov5


In [ ]:
!python3 detect.py --weights /content/drive/MyDrive/object_detection/weights/best.pt \
    --source /content/drive/MyDrive/object_detection/colab/danger_objects_v1_0/images/ \
    --img 640 --save-txt --project /content/drive/MyDrive/object_detection/colab/yolov5/runs/danger_detection_v1_0 --name latest_results --exist-ok

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
detect: weights=['/content/drive/MyDrive/KNU_Project2/object_detection/weights/best.pt'], source=/content/drive/MyDrive/KNU_Project2/object_detection/colab/danger_objects_v1_0/images/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/KNU_Project2/object_detection/colab/yolov5/runs/danger_detection_v1_0, name=latest_results, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, v

In [ ]:
import os
import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def box_area(box):
  return (box[2]-box[0])*(box[3]-box[1])


def box_iou_calc(boxes1, boxes2):
  area1 = box_area(boxes1.T)
  area2 = box_area(boxes2.T)


  lt = np.maximum(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
  rb = np.minimum(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]


  inter = np.prod(np.clip(rb - lt, a_min=0, a_max=None), 2)
  return inter / (area1[:, None] + area2 - inter)  # iou = inter / (area1 + area2 - inter)

In [ ]:
def process_batch(confusion_matrix, labels, detections, num_classes, iou_threshold=0.5):
  """
  Return intersection-over-union (Jaccard index) of boxes.
  Both sets of boxes are expected to be in (x1, y1, x2, y2) format.
  Arguments:
      detections (Array[N, 5]), class, x1, y1, x2, y2
      labels (Array[M, 5]), class, x1, y1, x2, y2
  Returns:
      None, updates confusion matrix accordingly
  """
  gt_classes = labels[:, 0].astype(np.int16)

  local_confusion_matrix = np.zeros((num_classes+1, num_classes+1), dtype=np.int32)


  """try:
      detections = detections[detections[:, 4] > self.CONF_THRESHOLD]
  except IndexError or TypeError:
      # detections are empty, end of process
      for i, label in enumerate(labels):
          gt_class = gt_classes[i]
          self.matrix[self.num_classes, gt_class] += 1
      return"""


  detection_classes = detections[:, 0].astype(np.int16)


  all_ious = box_iou_calc(labels[:, 1:], detections[:, 1:])
  want_idx = np.where(all_ious > iou_threshold)


  all_matches = [[want_idx[0][i], want_idx[1][i], all_ious[want_idx[0][i], want_idx[1][i]]]
                  for i in range(want_idx[0].shape[0])]

  all_matches = np.array(all_matches)
  if all_matches.shape[0] > 0:  # if there is match
      all_matches = all_matches[all_matches[:, 2].argsort()[::-1]]


      all_matches = all_matches[np.unique(all_matches[:, 1], return_index=True)[1]]


      all_matches = all_matches[all_matches[:, 2].argsort()[::-1]]


      all_matches = all_matches[np.unique(all_matches[:, 0], return_index=True)[1]]


  for i, label in enumerate(labels):
      gt_class = gt_classes[i]
      if all_matches.shape[0] > 0 and all_matches[all_matches[:, 0] == i].shape[0] == 1:
          detection_class = detection_classes[int(all_matches[all_matches[:, 0] == i, 1][0])]
          local_confusion_matrix[detection_class, gt_class] += 1
      else:
          local_confusion_matrix[num_classes, gt_class] += 1


  for i, detection in enumerate(detections):
      if not all_matches.shape[0] or ( all_matches.shape[0] and all_matches[all_matches[:, 1] == i].shape[0] == 0 ):
          detection_class = detection_classes[i]
          local_confusion_matrix[detection_class, num_classes] += 1

  return local_confusion_matrix

In [ ]:
original_test_data_label_path   = '/content/drive/MyDrive/object_detection/colab/danger_objects_v1_0/labels/'
prediction_test_data_label_path = '/content/drive/MyDrive/object_detection/colab/yolov5/runs/danger_detection_v1_0/latest_results/labels/'

confusion_matrix = np.zeros((5, 5), np.int32)

original_label_list = glob.glob(os.path.join(original_test_data_label_path, '*.txt'))
for original_label_filename in original_label_list:
  prediction_label_filename = os.path.join(prediction_test_data_label_path, os.path.basename(original_label_filename))

  # read original label from file
  init_labels = []
  with open(original_label_filename, 'r') as f:
    lines = f.readlines()
    for line in lines:
      data = line.split(' ')
      cls_idx = int(data[0])
      x = float(data[1])
      y = float(data[2])
      w = x+float(data[3])
      h = y+float(data[4])
      init_labels.append([cls_idx, x, y, w, h])

  pred_labels = []
  if os.path.exists(prediction_label_filename):
    with open(prediction_label_filename) as f:
      lines = f.readlines()
      for line in lines:
        data = line.split(' ')
        cls_idx = int(data[0])
        x = float(data[1])
        y = float(data[2])
        w = x+float(data[3])
        h = y+float(data[4])
        pred_labels.append([cls_idx, x, y, w, h])


    init_labels = np.array(init_labels)
    pred_labels = np.array(pred_labels)
    batch_confusion_matrix = process_batch(confusion_matrix, init_labels, pred_labels, 4, 0.5)
    confusion_matrix = confusion_matrix + batch_confusion_matrix
  else:
    #test
    for init_label in init_labels:
      confusion_matrix[4][init_label[0]] = confusion_matrix[4][init_label[0]]+1

print(confusion_matrix)

[[287   0   0   0   0]
 [  0 265   0   0   1]
 [  0   0 278   0  10]
 [  0   0   2 233   1]
 [  1   4   3  14   0]]


In [ ]:
tp = np.trace(confusion_matrix)
total = np.sum(confusion_matrix, (0, 1))
print("Accuracy:", tp/total * 100)

Accuracy: 96.7242948134668
